## Dataset generation from deformable mirror

A Parabolic GRIN fiber or a Step-Index fiber can be used

### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from mmfsim.grid import Grid
from mmfsim.fiber import GrinFiber, StepIndexFiber
from mmfsim.datasets import SimulatedGrinSpeckleOutputDataset, SimulatedStepIndexSpeckleOutputDataset

### Setup

Set the computation grid and the fiber characteristics

In [ ]:
fiber_type: str = 'grin' # 'grin' or 'step'
grid_settings: dict = {'pixel_size': 0.25e-6, 'pixel_numbers': (256, 256)}
fiber_settings: dict = {'radius': 26e-6, 'wavelength': 435e-9, 'n1': 1.465, 'n2': 1.45}
dataset_settings: dict = {'length': 1000, 'degen': True, 'phases_dim': (20,20)}
fourier_settings: dict = {'pad': 1}
export_settings: dict = {'return_output_fields': True, 'add_exp_noise': False, 'noise_func': np.max, 'file_type': 'hdf5'}

### Dataset generation

Generate the grid and the fiber

In [ ]:
grid = Grid(**grid_settings)

if fiber_type.lower() in ['grin']:
    fiber = GrinFiber(**fiber_settings)
elif fiber_type.lower() in ['step', 'si']:
    fiber = StepIndexFiber(**fiber_settings)
else:
    raise ValueError(f"Unknown fiber type.")
print(fiber)

Compute and store the modes for a much faster dataset generation, plot one for checking extent

In [ ]:
fiber.compute_modes_fields(grid)
plt.imshow(fiber._modes[:,:,1,-1])

Compute dataset

In [ ]:
dset = SimulatedGrinSpeckleOutputDataset(fiber, grid, N_modes=fiber._N_modes_degen, **dataset_settings)
dset.compute_from_transfer_matrix(phases_dim=dataset_settings['phases_dim'])
dset.compute_fourier_transforms(**fourier_settings)

### Dataset checking and export

Display a random draw from the dataset as a check

In [ ]:
rand_idx = np.random.randint(0, dset._fields.shape[-1])

fig, axs = plt.subplots(1, 2, figsize=(15, 5))
nf = axs[0].imshow(dset.intensities[..., rand_idx], cmap='gray')
ff = axs[1].imshow(np.square(np.abs(dset._transf[..., rand_idx])), cmap='gray')
plt.colorbar(nf, ax=axs[0], fraction=0.046, pad=0.04)
plt.colorbar(ff, ax=axs[1], fraction=0.046, pad=0.04)
axs[0].set_title('Near field intensity')
axs[1].set_title('Far field intensity')

Export it if it looks ok

In [ ]:
dset.export(**export_settings)